<a href="https://colab.research.google.com/github/ksetdekov/LLM_2024_education/blob/hw1/%D0%94%D0%971.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов (за каждый метод можно получить максимум 5 баллов).

Разбалловка:
*   **Воспроизводимость и читабельность кода - 1 балл** (все воспроизвелось и все понятно для проверяющего - 1 балл; непонятный код и/или ничего не воспроизвелось - 0 баллов).
*   **Корректность метода - 1 балл** (метод математически корректен и с точки зрения логики кода нет ошибок - 1 балл; все остальные случаи - 0 баллов).
*   **Описание метода в техническом отчете - 2 балла** (есть подробное описание метода и почему используете именно его с обоснованиями - 2 балла; есть описание метода, но нет обоснования - 1 балл; нет описания метода- 0 баллов).
*   **Иновационность - 1 балл** (используете нетривиальную обработку/лосс-функцию/модель - 1 балл; просто перебираете модели из коробок - 0 баллов).

!!! ДЗ необходимо выполнять только в Google Colab !!!

Присылать на почту llmrisks@yandex.ru с номером ДЗ и ФИО в теме. Каждая ДЗ отдельным письмом с отдельной темой.

# 1. Информация о сабмите

ФИО - **Сетдеков Кирилл Раильвич**

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*

*Балл за «инновационность» будет присваиваться на основе содержания этой части. Если ваше отличие от бейзлайна это просто почистить тексты от стоп-слов или поменять одну базовую модель для построения word embeddings на другую, этот балл будет 0. Пробуйте разные подходы, модели, экспериментируйте с предварительной обработкой, параметрами и т. д. Можно переопределить уже существующий подход. Это нормально, что некоторые из ваших экспериментов не сработали так, как вы ожидали. Покажите нам, что вы проявили творческий подход и провели несколько экспериментов.*

Пример решения на полный балл для похожей задачи:

# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [6]:
!python --version

Python 3.10.12


In [2]:
# !pip3 install --upgrade nltk gensim bokeh numpy~=1.19.5 scikit-learn~=0.24.2
!pip install --upgrade nltk gensim bokeh numpy>=1.24.4 scikit-learn>=1.3.1

import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api
# and some other your dependencies


## 3.2 Загрузка данных

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

data = list(open("./quora.txt", encoding="utf-8"))

--2024-12-10 09:22:19--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-12-10 09:22:20--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc39bce1292356e4d3003787284b.dl.dropboxusercontent.com/cd/0/inline/Cf82QvbwTLNVKiIMF2aChN5DNA4VTFWfQTeYKodjt01cbmB_V5UfsGtE6UPBFA0RuhVScMuG_rWLxFkJhE8mcuVhBVkdr8UNqtKS0RHlEP7g-pKInz1q-CyuGHg9whotdbQ/file?dl=1# [following]
--2024-12-10 09:22:21--  https://uc39bce1292356e4d3003787284b.dl.dropboxusercontent.com/cd/0/inline/Cf82Qvbw

## 3.3. Модель

In [4]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([model.vector_size], dtype='float32')
    word_vectors = []

    data_tok = tokenizer.tokenize(phrase.lower())
    word_vectors = np.array([model.get_vector(word) for word in data_tok if word in model.key_to_index.keys()])

    if word_vectors.size != 0:
        vector = np.average(word_vectors, axis=0)

    return vector

In [5]:
tokenizer = WordPunctTokenizer()
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [8]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

## 3.4. Применение модели

In [25]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    from scipy.spatial.distance import cosine
    from tqdm.notebook import tqdm
    emb = get_phrase_embedding(query)
    similarities = np.array([1 - cosine(emb, embedding) for embedding in tqdm(data_vectors)])
    indices = np.argpartition(-similarities, range(10))[:10]

    return [data[ind] for ind in indices]

In [26]:
find_nearest(query="How does Trump?", k=10)

  0%|          | 0/537272 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [12]:
# Test case: Geographical question with specific terms
query = "What is the capital of France?"
results = find_nearest(query=query, k=10)

# Print results
print("Query:", query)
print("\nMost similar lines:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.strip()}")


100%|██████████| 537272/537272 [00:13<00:00, 39579.58it/s]


Query: What is the capital of France?

Most similar lines:
1. What is the capital city of France?
2. What is the capital of uk?
3. What is the capital of England?
4. What is the capital of Denmark?
5. What is the capital of America?
6. What is the capital of India?
7. What is the capital of Finland?
8. What is the capital of Argentina?
9. What is the capital of Singapore?
10. What is the capital of Mexico?


# вариант улучшения 1 - Использование взвешенного усреднения эмбеддингов

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm

# Step 1: Precompute TF-IDF weights for all words in the dataset
tokenizer = WordPunctTokenizer()
corpus = [" ".join(tokenizer.tokenize(line.lower())) for line in data]  # Tokenize and normalize the dataset

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)  # Fit TF-IDF on the entire dataset
tfidf_vocab = vectorizer.vocabulary_  # Extract the vocabulary
tfidf_weights = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))  # Get TF-IDF scores per word

# Step 2: Adjust get_phrase_embedding to use TF-IDF weights
def get_phrase_embedding_with_tfidf(phrase):
    """
    Convert phrase to a vector by aggregating its word embeddings with TF-IDF weights.
    """
    vector = np.zeros([model.vector_size], dtype='float32')
    word_vectors = []
    weights = []

    # Tokenize phrase and calculate embeddings
    data_tok = tokenizer.tokenize(phrase.lower())
    for word in data_tok:
        if word in model.key_to_index.keys() and word in tfidf_vocab:
            word_vectors.append(model.get_vector(word))
            weights.append(tfidf_weights[word])  # Use precomputed TF-IDF weights

    if word_vectors:  # If embeddings are found
        word_vectors = np.array(word_vectors)
        weights = np.array(weights)
        vector = np.average(word_vectors, axis=0, weights=weights)  # Weighted average

    return vector

# Step 3: Compute vector embeddings for all lines in the dataset using the new function
data_vectors_tfidf = np.array([get_phrase_embedding_with_tfidf(l) for l in tqdm(data)])


# Step 4: Adjust find_nearest to work with the new embeddings
def find_nearest_with_tfidf(query, k=10):
    """
    Find the k most similar lines to the query using TF-IDF weighted embeddings.
    """
    from scipy.spatial.distance import cosine
    emb = get_phrase_embedding_with_tfidf(query)
    similarities = np.array([1 - cosine(emb, embedding) for embedding in tqdm(data_vectors_tfidf, ncols=100)])

    indices = np.argpartition(-similarities, range(k))[:k]

    return [data[ind] for ind in indices]




 58%|█████▊    | 312423/537272 [08:46<06:18, 593.29it/s]  


  0%|          | 0/537272 [00:00<?, ?it/s]

In [22]:
# Test case: Geographical question with specific terms
query = "What is the capital of France?"
results = find_nearest_with_tfidf(query=query, k=10)

# Print results
print("Query:", query)
print("\nMost similar lines:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.strip()}")


  0%|                                                                    | 0/537272 [00:00<?, ?it/s]

Query: What is the capital of France?

Most similar lines:
1. What is the capital city of France?
2. What is the capital of uk?
3. What is the capital of U.K?
4. What is the capital of India?
5. Where is the capital of India?
6. What is the capital of England?
7. What is the capital of the U.K.?
8. What is the best university in France?
9. What is the capital city of Belgium?
10. What is the capital of America?
